<a href="https://colab.research.google.com/github/jsmsscd/3d-bioprinter/blob/main/Univ_Recomendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import required Libraries

# New Section

In [ ]:
import pandas as pd
import requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_excel("/content/drive/MyDrive/Directory of Institution - University 1604202302451437PM.xlsx")
#df = pd.read_csv('https://aishe.gov.in/aishe/collegeDirectoryIndex?hasReportLink=index')
df.head()

EDa on Data

In [ ]:
print(df.shape)
print(df.describe())
print(df.info())

In [ ]:
print(df.isnull().sum())

In [ ]:
if len(df[df.duplicated()]) > 0:
    print("No. of duplicated entries: ", len(df[df.duplicated()]))
    print(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head())
else:
    print("No duplicated entries found")

Imputation

In [ ]:
df['Location'].mode()[0]

In [ ]:

df['Location'].fillna(df['Location'].mode()[0],inplace=True)
df['Year When Decalred University'].fillna(df['Year When Decalred University'].mode()[0],inplace=True)
df['Specialised in'].fillna(df['Specialised in'].mode()[0],inplace=True)
print(df.isnull().sum())


In [ ]:
#df[‘Age’].fillna(train[‘Age’].mode(), inplace = True)
#df[‘Embarked’].fillna(train[‘Embarked’].mode(), inplace = True)

df.fillna(0,inplace=True)
print(df.isnull().sum())
print(df.shape)

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
for col in df.select_dtypes(include='object').columns:
  fig=sns.catplot(x=col, kind="count", data=df)
  fig.set_xticklabels(rotation=90)
  plt.show()

In [ ]:
df.info()

Popularaity based recomendation system

Based on Institutes added in survey year ,the istitute which is added first is old and a transperent institue.
  

In [ ]:
numratingdf=df.groupby('University Name').count()['The Institute Added In Survey Year'].reset_index()
numratingdf.rename(columns={'The Institute Added In Survey Year':'Number of times University added for survey'},inplace=True)
numratingdf

In [ ]:
avgratingdf=df.groupby('University Name').mean()['The Institute Added In Survey Year'].reset_index()
avgratingdf.rename(columns={'The Institute Added In Survey Year':'Avg Rating'},inplace=True)
avgratingdf

In [ ]:
popularity_df=numratingdf.merge(avgratingdf,on="University Name")
popularity_df

In [ ]:
popularity_df[popularity_df["Avg Rating"]>2020].sort_values('Avg Rating',ascending=False).head(50)

Collaborative filtering based Recomendation System

In [ ]:
df.info()

In [ ]:
rankdf=pd.read_excel("/content/drive/MyDrive/Univ ranking NIRF.xlsx")
rankdf.head(10)

In [ ]:
print(rankdf.info())


In [ ]:
rankdf.rename(columns={"Name":"University Name"},inplace=True)
rankdf.head()

In [ ]:
rankdf.shape

In [ ]:
univwithrankdf=df.merge(rankdf,on="University Name")
univwithrankdf.head()

We are not getting any record common in rankdf (univ details from NIRF ) and  df(Univ list from AISHE ) thats whay univwithrankdf is blank so we are going to add ranking feature in df(Univ list from AISHE )

Feature Engineering

In [ ]:
df.head()

In [ ]:
df['Rating']=df["The Institute Added In Survey Year"] -df["Year When Decalred University"]
df.head()

In [ ]:
df['Rating']+= df.apply(lambda x: 1 if x.Location == 'Urban' else 0.5, axis=1) 
df.head()

The univerity with lowes rating vlaue is the best one because as fast university added in survey is the univ which gain name in samll span of time thats whay it is added in survey as well as Urban univ can provide much better facilaties as compared to rural univs so thats why .5 is added in rating feature for rural and 1 for urban univs. 

Colaborative Filtering based Recomendation System

In [ ]:
sd=df.groupby(['State','District']).count()['Rating']
sd

Only those university which are added for survey in less than 5 years

In [ ]:
x=df.groupby('University Name').count()['Rating']<3
x[x]

In [ ]:
filtered_univ=x[x].index
filtered_univ

In [ ]:

final_df=df[df['University Name'].isin(filtered_univ)]
final_df

In [ ]:
pt=final_df.pivot_table(index='University Name',columns=['State','District'],values='Rating')

In [ ]:
pt.shape

In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

The data is in piviot table fromat 
Every univ can be represeted as a vector in the from of 319 dismensional space 
Now find similar vectors for this  we find eucledian distance of every vector with each other vector and based on distance we can find out similarity of univ

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores=cosine_similarity(pt)

In [ ]:
similarity_scores[1]

this is similarity score of first univ with other 318 univs


In [ ]:
from pandas.io.pytables import Index
def recommender(univ_name):
  #fetch index from univ name
  index=np.where(pt.index==univ_name)[0][0]
  print(index)
  # in similarity scores find the most similar items
  similar_items=sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:6]   
  for i in similar_items:
    print(pt.index[i[0]])
 

In [ ]:
recommender('Jawaharlal Nehru Technological University, Anantapur (Id: U-0016)')
